## Analisis de la Tasa de Homicidios en EEUU

In [2]:
# importamos librerias
import empiricaldist
import janitor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as ss

### 1. Asignamos una apariencia general a los graficos

In [5]:
# muestra los graficos y los guarda en el Notebooks
%matplotlib inline

sns.set_style(style='whitegrid') #parametros del estilo general de los graficos
sns.set_context(context='notebook') #parametros de escala de los elementos del grafico
plt.rcParams['figure.figsize'] = (11, 9.4) #personaliza las propiedades y estilos de los graficos de matplotlib


### 2. Cargar Datos

In [6]:
state = pd.read_csv('state.csv')
state.head(8)

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA
5,Colorado,5029196,2.8,CO
6,Connecticut,3574097,2.4,CT
7,Delaware,897934,5.8,DE


### 3. Tipos de datos de cada variable

In [7]:
state.dtypes

State            object
Population        int64
Murder.Rate     float64
Abbreviation     object
dtype: object

Tanto states como Abbreviation son de datos categoricos de tipo string. Population es de tipo entero y Murder.Rate is numerico